# Video Intra-Predictive Coding (IPP)

## Group Composition
- Víctor Fernández Díaz
- Cristhian Ceballos Moreno
- Pablo Gómez Rivas

## Theoretical Explanation
This notebook implements a hybrid video codec based on **inter-frame prediction**. The main objective is to reduce temporal and spatial redundancy in the video using an IPP scheme (one Intra-type frame followed by several Predictive-type frames).

The algorithm operates by dividing the video into Groups of Pictures (GOP). The first frame of each group is encoded as an independent image (**I-Frame**), while the subsequent ones (**P-Frames**) are generated by predicting motion from the previous frame. Instead of storing the full frame, only the **residual** (the difference between the original and the prediction) and the **motion vectors** are stored, achieving drastic data compression.

### Motion Vector Calculation
The calculation is performed using a **dense optical flow** technique combined with block averaging:
1. The **Farneback** algorithm (`cv.calcOpticalFlowFarneback`) is used to calculate the motion of each pixel between the previous reconstructed reference image (`prev_recon`) and the current image (`img_np`).
2. For each block of size `block_size` (e.g., 16x16), all motion vectors in that area are extracted and their **arithmetic mean** is calculated (`avg_vector = block_flow.mean(axis=0)`). This resulting vector represents the $(dx, dy)$ displacement of the block.

### Encoding and Disk Storage
Vectors are not saved frame-by-frame in individual files; instead, they are managed by **GOP (Group of Pictures)**:
* **Format:** The `pickle` library is used to serialize the data in binary format.
* **Storage:** Files are saved with the name `encoded_mv_chunk_XXXX.pkl`.
* **Structure:** Each file contains a list that includes the motion maps for all frames within an `intra_period`. This allows for efficient loading during decoding.

### I or P Block Decision (RD Optimization)
The algorithm uses a **Rate-Distortion (RD) Optimization** scheme to decide the block type:
* **Decision:** For each block, the encoder calculates the Lagrangian cost J=D+λR. It compares the cost of encoding as **Inter** (using the motion vector) versus **Intra** (using a constant value of 128). The mode with the lower cost J is selected.
* **Lambda (λ):** A parameter that balances distortion (D) and bitrate (R).
* **Storage:** This decision is stored implicitly in the `.pkl` file: `None` for Intra blocks and the `[dx, dy]` vector for Inter blocks.

### Visual Instrumentation
To analyze the codec's performance, the system can generate:
* **Prediction Frames:** The synthesized image before adding the residual.
* **Decision Maps:** A visual representation of where the RD optimization chose Intra (dark) vs Inter (light) blocks.

### Reference Image Calculation in the Compressor
To avoid accumulated error (**drift**), the compressor implements a **local decoding loop**:
1. After calculating the residual (difference between original and prediction) and encoding it, the compressor immediately **re-decodes** that residual.
2. The reference image (`prev_recon`) is generated by adding the prediction to the newly decoded residual.
3. This way, the compressor uses exactly the same reference image that the decoder will have as a reference, ensuring that the prediction is identical at both ends.

## Implementation

In [ ]:
%%writefile ../src/IPP.py
'''IPP... coding: runs an intra-predictive 2D image codec for each frame of a video sequence.'''

import sys
import io
import os
import logging
import numpy as np
import cv2 as cv
import av
from PIL import Image
import importlib
import re
import pickle
import glob
with open("/tmp/description.txt", "w") as f:  # Used by parser.py
    f.write(__doc__)
import parser
import main
import entropy_video_coding as EVC

# ------------------------------------------------------------
# Encoder / Decoder arguments
# ------------------------------------------------------------
parser.parser_encode.add_argument(
    "-T", "--transform", type=str,
    help=f"2D-transform, default: {EVC.DEFAULT_TRANSFORM}",
    default=EVC.DEFAULT_TRANSFORM
)
parser.parser_encode.add_argument(
    "-N", "--number_of_frames", type=parser.int_or_str,
    help=f"Number of frames to encode (default: {EVC.N_FRAMES})",
    default=f"{EVC.N_FRAMES}"
)
parser.parser_encode.add_argument(
    "--intra_period", type=int,
    help="Insert an intra frame every intra_period frames (default: 32)",
    default=32
)
parser.parser_encode.add_argument(
    "--block_size", type=int,
    help="Block size for motion compensation (default: 16)",
    default=16
)
parser.parser_encode.add_argument(
    "--rd_optimization", action="store_true",
    help="Enable Rate-Distortion optimization for block types (default: False)"
)
parser.parser_encode.add_argument(
    "--visualize_prediction", action="store_true",
    help="Save prediction images and I/P block maps (default: False)"
)
parser.parser_encode.add_argument(
    "--lamb", type=float,
    help="Lambda parameter for RD optimization (default: 0.5)",
    default=0.5
)

parser.parser_decode.add_argument(
    "-T", "--transform", type=str,
    help=f"2D-transform, default: {EVC.DEFAULT_TRANSFORM}",
    default=EVC.DEFAULT_TRANSFORM
)
parser.parser_decode.add_argument(
    "-N", "--number_of_frames", type=parser.int_or_str,
    help=f"Number of frames to decode (default: {EVC.N_FRAMES})",
    default=f"{EVC.N_FRAMES}"
)
parser.parser_decode.add_argument(
    "--intra_period", type=int,
    help="Insert an intra frame every intra_period frames (default: 32)",
    default=32
)
parser.parser_decode.add_argument(
    "--block_size", type=int,
    help="Block size for motion compensation (default: 16)",
    default=16
)
parser.parser_decode.add_argument(
    "--rd_optimization", action="store_true",
    help="Enable Rate-Distortion optimization for block types (default: False)"
)

args = parser.parser.parse_known_args()[0]

# ------------------------------------------------------------
# Import 2D transform codec
# ------------------------------------------------------------
if __debug__ and args.debug:
    print(f"IPP: Importing {args.transform}")

try:
    transform = importlib.import_module(args.transform)
except ImportError as e:
    print(f"Error: Could not find {args.transform} module ({e})")
    print(f"Make sure '2D-{args.transform}.py' is in the same directory as IPP.py")
    sys.exit(1)

# ------------------------------------------------------------
# IPP CoDec
# ------------------------------------------------------------
class CoDec(EVC.CoDec):
    def __init__(self, args):
        logging.debug("trace")
        super().__init__(args)
        self.transform_codec = transform.CoDec(args)
        logging.info(f"Using {args.transform} codec for IPP")
        logging.info(f"Intra period = {args.intra_period}")
        self.block_size = args.block_size
        self.mv_blocks = []
        self.total_mse = 0.0
        self.total_pixels_per_frame = 0

    def _save_mv_chunk(self, current_img_idx):
        """Saves the current chunk of motion vectors to a separate file (one per GOP)."""
        chunk_idx = current_img_idx // self.args.intra_period
        mv_file = f"{EVC.ENCODE_OUTPUT_PREFIX}_mv_chunk_{chunk_idx:04d}.pkl"

        # Calculate which vectors belong to this chunk
        start = chunk_idx * self.args.intra_period
        chunk_data = self.mv_blocks[start : current_img_idx + 1]

        with open(mv_file, 'wb') as f:
            pickle.dump(chunk_data, f)
        logging.info(f"Saved MV chunk to {mv_file}")

    # --------------------------------------------------------
    # Encoding
    # --------------------------------------------------------
    def encode(self):
        logging.debug("trace")
        fn = self.args.original
        logging.info(f"Encoding {fn}")
        container = av.open(fn)
        img_counter = 0
        exit_flag = False
        prev_recon = None

        for packet in container.demux():
            if __debug__:
                self.total_input_size += packet.size

            for frame in packet.decode():
                img = frame.to_image().convert("RGB")
                img_np = np.array(img, dtype=np.int16)
                raw_fn = f"/tmp/original_%04d.png" % img_counter
                code_fn = f"{EVC.ENCODE_OUTPUT_PREFIX}_%04d" % img_counter
                recon_fn = f"/tmp/recon_%04d.png" % img_counter
                res_fn = f"/tmp/residual_%04d.png" % img_counter
                img.save(raw_fn)

                if img_counter == 0 or (img_counter % self.args.intra_period) == 0:
                    # Intra frame
                    logging.info("Encoding intra frame")
                    O_bytes = self.transform_codec.encode_fn(raw_fn, code_fn)
                    self.transform_codec.decode_fn(code_fn, recon_fn)
                    recon = np.array(Image.open(recon_fn), dtype=np.int16)
                    self.mv_blocks.append(None)
                else:
                    # Inter frame con Motion Compensation + RD Optimization
                    logging.info("Encoding inter frame with RD optimization")
                    prev_gray = cv.cvtColor(np.clip(prev_recon, 0, 255).astype(np.uint8), cv.COLOR_RGB2GRAY)
                    curr_gray = cv.cvtColor(np.clip(img_np, 0, 255).astype(np.uint8), cv.COLOR_RGB2GRAY)
                    flow = cv.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 0.5, 3, 15, 3, 5, 1.2, 0)

                    H, W = img_np.shape[:2]
                    b_h, b_w = self.block_size, self.block_size
                    mv_frame = []
                    predicted = np.zeros_like(img_np, dtype=np.float32)

                    for y in range(0, H, b_h):
                        row = []
                        for x in range(0, W, b_w):
                            y0, y1 = y, min(y+b_h, H)
                            x0, x1 = x, min(x+b_w, W)

                            # Estimate motion vector and generate inter-prediction block
                            block_flow = flow[y0:y1, x0:x1]
                            avg_vector = block_flow.reshape(-1, 2).mean(axis=0)
                            dx, dy = avg_vector
                            ys = np.clip(np.arange(y0, y1) + dy, 0, H-1).astype(np.int32)
                            xs = np.clip(np.arange(x0, x1) + dx, 0, W-1).astype(np.int32)
                            p_block = np.zeros((y1-y0, x1-x0, 3), dtype=np.float32)
                            for c in range(3):
                                p_block[:,:,c] = prev_recon[np.ix_(ys, xs, [c])].reshape(y1-y0, x1-x0)

                            # RD Optimization: Decide between Intra (I) or Inter (P)
                            use_intra = False
                            if self.args.rd_optimization:
                                original_block = img_np[y0:y1, x0:x1]

                                # --- INTER (P) Evaluation ---
                                dist_inter = np.mean((original_block - p_block)**2)
                                # Estimate R: residual entropy + motion vector bits
                                rate_inter = np.log2(np.var(original_block - p_block) + 1) + 2 
                                j_inter = dist_inter + self.args.lamb * rate_inter

                                # --- INTRA (I) Evaluation ---
                                # No prediction used (baseline 128)
                                dist_intra = np.mean((original_block - 128)**2)
                                # Estimate R: original block entropy + intra overhead
                                rate_intra = np.log2(np.var(original_block) + 1) + 4 
                                j_intra = dist_intra + self.args.lamb * rate_intra

                                if j_intra < j_inter:
                                    use_intra = True
                            if use_intra:
                                # Encode as Intra block (no motion compensation)
                                predicted[y0:y1, x0:x1] = 128
                                row.append(None)
                            else:
                                # Encode as Inter block using motion vector
                                predicted[y0:y1, x0:x1] = p_block
                                row.append(avg_vector)
                        mv_frame.append(row)

                    self.mv_blocks.append(mv_frame)

                    # Visual instrumentation
                    if self.args.visualize_prediction:
                        # 1. Save the inter-frame prediction signal (P[n])
                        pred_img_save = np.clip(predicted, 0, 255).astype(np.uint8)
                        Image.fromarray(pred_img_save).save(f"/tmp/pred_{img_counter:04d}.png")
                        # 2. Generate I/P block decision map
                        # Create a segmentation map where dark blocks represent Intra mode 
                        # and light blocks represent Inter mode (motion compensated)
                        block_map = np.zeros((H, W), dtype=np.uint8)
                        for by, row in enumerate(mv_frame):
                            for bx, vec in enumerate(row):
                                y0, x0 = by * b_h, bx * b_w
                                y1, x1 = min(y0 + b_h, H), min(x0 + b_w, W)
                                # Decision based on RD optimization: None = Intra, vector = Inter
                                val = 50 if vec is None else 200
                                block_map[y0:y1, x0:x1] = val
                        map_fn = f"/tmp/map_{img_counter:04d}.png"
                        Image.fromarray(block_map).save(map_fn)
                        logging.info(f"Saved prediction and block map for frame {img_counter}")

                    residual = img_np - predicted
                    residual = np.clip(residual + 128, 0, 255).astype(np.uint8)
                    residual_img = Image.fromarray(residual, mode="RGB")
                    residual_img.save(res_fn)
                    O_bytes = self.transform_codec.encode_fn(res_fn, code_fn)
                    self.transform_codec.decode_fn(code_fn, recon_fn)
                    residual_dec = np.array(Image.open(recon_fn), dtype=np.int16)
                    recon = predicted + (residual_dec - 128)

                frame_mse = np.mean((img_np.astype(np.float32) - recon.astype(np.float32))**2)
                self.total_mse += frame_mse
                if self.total_pixels_per_frame == 0:
                    self.total_pixels_per_frame = img_np.shape[0] * img_np.shape[1]

                self.total_output_size += O_bytes
                prev_recon = recon

                # Save chunk if we reached the end of an intra_period or the last frame
                if (img_counter + 1) % self.args.intra_period == 0 or (img_counter + 1) == self.args.number_of_frames:
                    self._save_mv_chunk(img_counter)

                img_counter += 1
                logging.info(f"img_counter = {img_counter} / {args.number_of_frames}")
                if img_counter >= args.number_of_frames:
                    exit_flag = True
                    break
            if exit_flag:
                break

        self.N_frames = img_counter

        # J = R + D
        # 1. Calculate Rate (R): Bits per pixel including motion vectors
        mv_files = glob.glob(f"{EVC.ENCODE_OUTPUT_PREFIX}_mv_chunk_*.pkl")
        total_mv_bytes = sum(os.path.getsize(f) for f in mv_files)

        total_bits = (self.total_output_size + total_mv_bytes) * 8
        total_pixels_video = self.total_pixels_per_frame * self.N_frames
        R = total_bits / total_pixels_video

        # 2. Calculate Distortion (D): Root Mean Squared Error
        avg_mse = self.total_mse / self.N_frames
        D = np.sqrt(avg_mse)

        # 3. Efficiency Metric J
        J = D + (self.args.lamb * R)

        print("\n" + "="*40)
        print(" FINAL PERFORMANCE METRICS (J = D + λR)")
        print("="*40)
        print(f"Lambda (λ):     {self.args.lamb:.4f}")
        print(f"Rate (R):       {R:.4f} bits/pixel")
        print(f"Distortion (D): {D:.4f} (RMSE)")
        print(f"Efficiency (J): {J:.4f}")
        print("="*40 + "\n")
        print(f"Frames processed:     {self.N_frames}")
        print(f"Total Residual Bytes: {self.total_output_size}")
        print(f"Total MV Bytes:       {total_mv_bytes}")

        self.height, self.width = img_np.shape[:2]
        self.N_channels = img_np.shape[2]

    # --------------------------------------------------------
    # Decoding
    # --------------------------------------------------------
    def decode(self):
        logging.debug("trace")
        prev_recon = None
        current_chunk_idx = -1
        current_mv_chunk = []

        for img_counter in range(self.args.number_of_frames):
            # Dynamic loading: Load the correct MV chunk for the current GOP
            chunk_idx = img_counter // self.args.intra_period
            if chunk_idx != current_chunk_idx:
                mv_file = f"{EVC.ENCODE_OUTPUT_PREFIX}_mv_chunk_{chunk_idx:04d}.pkl"
                with open(mv_file, 'rb') as f:
                    current_mv_chunk = pickle.load(f)
                current_chunk_idx = chunk_idx
                logging.info(f"Loaded MV chunk {mv_file} for frame {img_counter}")

            code_fn = f"{EVC.ENCODE_OUTPUT_PREFIX}_%04d" % img_counter
            out_fn = f"{EVC.DECODE_OUTPUT_PREFIX}_%04d.png" % img_counter
            logging.info(f"Decoding frame {code_fn} into {out_fn}")
            self.transform_codec.decode_fn(code_fn, out_fn)
            img_np = np.array(Image.open(out_fn), dtype=np.int16)

            mv_frame = current_mv_chunk[img_counter % self.args.intra_period]
            if mv_frame is None:
                recon = img_np
            else:
                H, W = img_np.shape[:2]
                b_h, b_w = self.block_size, self.block_size
                predicted = np.zeros_like(img_np, dtype=np.float32)
                for by, row in enumerate(mv_frame):
                    for bx, vec in enumerate(row):
                        y0, x0 = by * b_h, bx * b_w
                        y1, x1 = min(y0 + b_h, H), min(x0 + b_w, W)
                        if vec is None:
                            # Reconstruct Intra block
                            predicted[y0:y1, x0:x1] = 128
                        else:
                            # Reconstruct Inter block using stored motion vector
                            dx, dy = vec
                            ys = np.clip(np.arange(y0, y1) + dy, 0, H-1).astype(np.int32)
                            xs = np.clip(np.arange(x0, x1) + dx, 0, W-1).astype(np.int32)
                            for c in range(img_np.shape[2]):
                                predicted[y0:y1, x0:x1, c] = prev_recon[np.ix_(ys, xs, [c])].reshape(y1-y0, x1-x0)

                # Combine prediction with decoded residual
                recon = predicted + (img_np - 128)

            recon = np.clip(recon, 0, 255).astype(np.uint8)
            Image.fromarray(recon, mode="RGB").save(out_fn)
            prev_recon = recon

# ------------------------------------------------------------
# Main
# ------------------------------------------------------------
if __name__ == "__main__":
    main.main(parser.parser, logging, CoDec)


## Usage within VCF

### Integration in the Compression Framework

**IPP** integrates into **VCF (Visual Coding Framework)** as a **hybrid video codec** that combines motion compensation with spatial transforms. Unlike static image codecs, its function is to exploit both **temporal redundancy** (between frames) and **spatial redundancy** (within the residual error).

#### Compression Flow with IPP:

1. **Video Sequence (Frames)**
2. **Intra/Inter Decision (RD Optimization)**
3. **Motion Estimation (Farneback Optical Flow) → [Motion Vectors]**
4. **Residual Calculation (Original - Predicted)**
5. **Spatial Transform (2D-DCT, 2D-DWT, etc.)**
6. **Quantization & Entropy Coding**
7. **Compressed Bitstream (Residuals + .pkl Motion Chunks)**

#### Advantages over Static Frame Coding:

1. **Temporal Efficiency**: Drastically reduces file size by only storing the "difference" (residues) and small motion vectors instead of full images.
2. **Error Control**: Uses a **local decoding loop** to generate reference frames, preventing the accumulation of reconstruction errors (*drift*).
3. **Hybrid Versatility**: Can wrap any existing VCF spatial transform (DCT, DWT, VQ) to compress the residual signal.

#### IPP Parameters in VCF:

| Parameter | Flag | Type | Description |
|-----------|------|------|-------------|
| Transform | `-T` | str | 2D-transform for residuals (default: 2D-DCT) |
| Intra Period | `--intra_period` | int | Frames between I-frames (default: 32) |
| Block Size | `--block_size` | int | Size for motion compensation (default: 16) |
| RD Optimization | `--rd_optimization`| bool | Enables Rate-Distortion block type decision |
| Lambda | `--lamb` | float | Weight for Rate-Distortion trade-off (default: 0.5) |
| Visualize | `--visualize_prediction` | bool | Saves prediction and block maps to `/tmp` |
| Frame Count | `-N` | int | Number of frames to process |

#### Example Usage:

```bash
# Basic encoding using default DCT
python IPP.py encode -o video.mp4

# High-efficiency encoding with DWT, RD optimization and specific Lambda
python IPP.py encode -T 2D-DWT -N 100 --block_size 8 --rd_optimization --lamb 0.5 -o video.mp4

# Decoding the sequence (must match transform and number of frames)
python IPP.py decode -T 2D-DWT -N 100 --intra_period 32 --block_size 8
```

### CoDec Class Implementation:
The CoDec class in IPP.py manages the video state and temporal dependencies:
- Encoder: Computes Farneback flow → Averages vectors per block → Performs RD search (Intra vs Inter) → Calculates and transforms the residual.
- Decoder: Loads .pkl motion chunks → Reconstructs prediction using stored vectors or "None" (Intra) → Adds the decoded residual.
- Output files: encoded_mv_chunk_XXXX.pkl (Motion) and encoded_XXXX (Residuals).

## Practical Usage Examples

In [ ]:
from IPython.display import Video

In [ ]:
#!pip install -r ../requirements.txt

### Help about basic functionality

In [ ]:
!python ../src/IPP.py -h

### Help to encode

In [ ]:
!python ../src/IPP.py encode -h

### Encode and decode a local video

In [ ]:
! [ -f /tmp/original.avi ] || wget http://www.hpca.ual.es/~vruiz/videos/coastguard_352x288x30x420x300.avi -O /tmp/original.avi
!ffplay -i /tmp/original.avi

In [ ]:
%%bash
rm -f /tmp/original_* /tmp/encoded* /tmp/decoded_* /tmp/pred_* /tmp/map_* /tmp/recon_* /tmp/residual_*
python ../src/IPP.py encode -N 100 --rd_optimization --lamb 0.5 --visualize_prediction  -o /tmp/original.avi
python ../src/IPP.py decode -N 100 --rd_optimization

In [ ]:
!ffplay -i /tmp/decoded_%04d.png -loop 0

### Encoding a remote video

In [ ]:
Video("http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4")

In [ ]:
%%bash
rm -f /tmp/original_* /tmp/encoded* /tmp/decoded_* /tmp/pred_* /tmp/map_* /tmp/recon_* /tmp/residual_*
python ../src/IPP.py encode -N 100 -o http://www.hpca.ual.es/~vruiz/videos/mobile_352x288x30x420x300.mp4
python ../src/IPP.py decode -N 100

In [ ]:
!ffplay -i /tmp/decoded_%04d.png -loop 0

### Using 2D-DWT

In [ ]:
%%bash
rm -f /tmp/original_* /tmp/encoded* /tmp/decoded_* /tmp/pred_* /tmp/map_* /tmp/recon_* /tmp/residual_*
python ../src/IPP.py encode -T 2D-DWT -o /tmp/original.avi
python ../src/IPP.py decode -T 2D-DWT

In [ ]:
!ffplay -i /tmp/decoded_%04d.png -loop 0

### Using VQ
Working on it ...

In [ ]:
%%bash
rm -f /tmp/original_* /tmp/encoded* /tmp/decoded_* /tmp/pred_* /tmp/map_* /tmp/recon_* /tmp/residual_*
python ../src/IPP.py encode -N 8 -a VQ -o /tmp/original.avi
python ../src/IPP.py decode -N 8 -a VQ 

In [ ]:
!ffplay -i /tmp/decoded_%04d.png -loop 0

### Low Bit-Rate Configuration (Maximum Compression)
Working on it ...

In [ ]:
%%bash
rm -f /tmp/original_* /tmp/encoded* /tmp/decoded_* /tmp/pred_* /tmp/map_* /tmp/recon_* /tmp/residual_*
python ../src/IPP.py encode -N 20 --intra_period 10 --block_size 64 -o /tmp/original.avi
python ../src/IPP.py decode -N 20 --intra_period 10 --block_size 64

In [ ]:
!ffplay -i /tmp/decoded_%04d.png -loop 0

### Visual Analysis

In [ ]:
!pip install ipywidgets

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
from ipywidgets import interact, IntSlider
import os
import glob
def plot_video_debug(frame_idx):
    orig_path = f"/tmp/original_{frame_idx:04d}.png"
    pred_path = f"/tmp/pred_{frame_idx:04d}.png"
    map_path = f"/tmp/map_{frame_idx:04d}.png"
    res_path = f"/tmp/residual_{frame_idx:04d}.png"
    if not os.path.exists(pred_path):
        if os.path.exists(orig_path):
            plt.figure(figsize=(5,5))
            plt.imshow(Image.open(orig_path))
            plt.title(f"Original (Frame {frame_idx} - INTRA)")
            plt.axis('off')
            plt.show()
        return
    fig, ax = plt.subplots(1, 4, figsize=(20, 5))
    ax[0].imshow(Image.open(orig_path))
    ax[0].set_title(f"Original ({frame_idx})")
    ax[1].imshow(Image.open(pred_path))
    ax[1].set_title("Prediction")
    if os.path.exists(res_path):
        ax[2].imshow(Image.open(res_path))
        ax[2].set_title("Residual")
    else:
        ax[2].text(0.5, 0.5, 'N/A', ha='center')
    ax[3].imshow(Image.open(map_path), cmap='gray')
    ax[3].set_title("Decision Map")
    for a in ax: a.axis('off')
    plt.tight_layout()
    plt.show()
num_frames = len(glob.glob("/tmp/original_*.png"))
if num_frames > 0:
    interact(plot_video_debug, frame_idx=IntSlider(min=0, max=num_frames-1, step=1, value=1))
else:
    print("No frames found in /tmp. Please run the encoder first.")